# Pathway similarity network

#### This notebooks generate the file adapted to [Cytoscape](http://www.cytoscape.org/)  to plot the pathway similarity network

Author: [Daniel Domingo-Fernández](https://github.com/ddomingof) 



In [1]:
import time
import sys
import os

import networkx as nx
import pandas as pd
from collections import defaultdict
import itertools as itt
import json

from bio2bel_kegg.manager import Manager as KeggManager
from bio2bel_reactome.manager import Manager as ReactomeManager
from bio2bel_wikipathways.manager import Manager as WikiPathwaysManager


In [2]:
time.asctime()

'Mon Feb 26 09:08:57 2018'

In [3]:
print(sys.version)

3.4.5 (default, Dec 11 2017, 14:22:24) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-16)]


In [4]:
#Location of the cloned repo
BASE_PATH = os.environ['COMPATH']
EXCELS_PATH = os.path.join(BASE_PATH,'src','compath','static','resources','excel')


In [5]:
kegg_excel = os.path.join(EXCELS_PATH,'kegg_gene_sets.csv')
reactome_excel= os.path.join(EXCELS_PATH,'reactome_gene_sets.csv')
wikipathways_excel = os.path.join(EXCELS_PATH,'wikipathways_gene_sets.csv')

In [6]:
def create_pathway_gene_set_dict(dataframe):
    """Creates a pathway genes dictionary
    
    :param pandas.DataFrame dataset: gene sets df
    :rtype: collections.defaultdict
    :returns: dictionary of pathway gene sets
    """
    
    pathway_dictionary = defaultdict(set)
    
    for pathway_name in dataframe: # iterate over columns in dataframe

        for gene in dataframe[pathway_name].unique():
            if not isinstance(gene, str): # There are NaN in the Pandas nArray
                continue

            pathway_dictionary[pathway_name].add(gene)
            
    return pathway_dictionary

Load KEGG

In [7]:
kegg_dataframe = pd.read_csv(kegg_excel, dtype=object)

# Remove the 'Homo sapiens' out of the KEGG pathways
kegg_dataframe.columns = [
    kegg_pathway.replace(' - Homo sapiens (human)', '')
    for kegg_pathway in kegg_dataframe
] 

kegg_pathways = create_pathway_gene_set_dict(kegg_dataframe)

kegg_manager = KeggManager()

kegg_names_to_ids = kegg_manager.get_pathway_names_to_ids()

try:
    assert (len(kegg_names_to_ids.keys()) == 323)
except AssertionError as error:
    print('KEGG Database contains {} pathways'.format(len(kegg_names_to_ids.keys())))
    
try:
    assert (len(kegg_pathways.keys()) == 323)
except AssertionError as error:
    print('DataFrame does not contain 323 pathways, contains: {}'.format(len(kegg_pathways.keys())))

KEGG Database contains 324 pathways


Load Reactome

In [8]:
reactome_dataframe = pd.read_csv(reactome_excel, dtype=object)

reactome_pathways = create_pathway_gene_set_dict(reactome_dataframe)

reactome_manager = ReactomeManager()

reactome_names_to_ids = reactome_manager.get_pathway_names_to_ids()

try:
    assert (len(reactome_names_to_ids.keys()) == 2162)
except AssertionError as error:
    print('Reactome Database contains {} pathways'.format(len(reactome_names_to_ids.keys()))) # Total of 2662 of those: 2132 are not empty
    
try:
    assert (len(reactome_pathways.keys()) == 2132)
except AssertionError as error:
    print('DataFrame does not contain 2132 pathways, contains: {}'.format(len(reactome_pathways.keys())))


Load WikiPathways

In [13]:
wikipathways_dataframe = pd.read_csv(wikipathways_excel, dtype=object)

wikipathways_pathways = create_pathway_gene_set_dict(wikipathways_dataframe)

wikipathways_manager = WikiPathwaysManager()

wikipathways_names_to_ids = wikipathways_manager.get_pathway_names_to_ids()

try:
    assert (len(wikipathways_names_to_ids.keys()) == 417)
except AssertionError as error:
    print('WikiPathways Database contains {} pathways'.format(len(wikipathways_names_to_ids.keys())))
    
try:
    assert (len(wikipathways_pathways.keys()) == 408)
except AssertionError as error:
    print('DataFrame does not contain 408 pathways, contains: {}'.format(len(wikipathways_pathways.keys())))

Create network

In [17]:
KEGG = 'kegg'
REACTOME = 'reactome'
WIKIPATHWAYS = 'wikipathways'

KEGG_URL = 'http://www.kegg.jp/kegg-bin/show_pathway?map=map{}&show_description=show'
REACTOME_URL = 'https://reactome.org/PathwayBrowser/#/{}'
WIKIPATHWAYS_URL = 'https://www.wikipathways.org/index.php/Pathway:{}'


def export_cytoscape_js(graph):
    """Convert networkx to Cytoscape.js JSON format
    
    :param: networkx.Graph graph: graph
    :rtype: dict
    :return: dictionary representing the json object in cytoscape js
    """    
    network_dict = defaultdict(list)
    
    info_to_id = {} # linking pathway info tuple to identifier
    
    for node_id, node in enumerate(graph.nodes()):
        
        info_to_id[node] = node_id
        
        node_object = {}
        node_object["data"] = {}
        node_object["data"]["id"] = node_id
        node_object["data"]["name"] = node[0]
        node_object["data"]["resource"] = node[1]
        
        
        if node[1] == REACTOME:
            node_object["data"]["url"] = REACTOME_URL.format(reactome_names_to_ids[node[0]])
            
        elif node[1] == KEGG:
            node_object["data"]["url"] = KEGG_URL.format(kegg_names_to_ids[node[0]])

        elif node[1] == WIKIPATHWAYS:
            node_object["data"]["url"] = WIKIPATHWAYS_URL.format(wikipathways_names_to_ids[node[0]])
            
        else:
            print('Error with {}. Check the resource assignment. Make sure all pathway databases are populated'.format(node))
            break        
        
        # TODO: Add link identifier reactome/kegg/wikipathways
        network_dict["nodes"].append(node_object.copy())
        

    for source, target, data in graph.edges(data=True):
        
        nx = {}
        nx["data"]={}
        nx["data"]["source"]=info_to_id[source]
        nx["data"]["target"]=info_to_id[target]
        nx["data"]["similarity"]=data['similarity']
        
        network_dict["edges"].append(nx)
        
    return network_dict


def populate_similarity_network(graph, threshold, pathway_resource_1, gene_sets_1, pathway_resource_2, gene_sets_2):
    """Populates the similarity pathway network in place given two gene sets a threshold
    
    :param networkx.Graph graph: similarity pathway network
    :param float threshold: percentage of the minimum threshold to add the link to the network (e.g., 0.75 means 75% similarity)
    :param str pathway_resource_1: name of the resource 1
    :param dict gene_sets_1: pathway gene set dict 2
    :param str pathway_resource_2: name of the resource 2
    :param dict gene_sets_2: pathway gene set dict 2
    """
    
    for pathway_1, pathway_2 in itt.product(gene_sets_1.keys(), gene_sets_2.keys()):
        
        if (pathway_1,pathway_resource_1) == (pathway_2,pathway_resource_2):
            continue

        intersection = len(gene_sets_1[pathway_1].intersection(gene_sets_2[pathway_2]))
        smaller_set = min(len(gene_sets_1[pathway_1]), len(gene_sets_2[pathway_2]))

        similarity = float(intersection/smaller_set) # Formula to calculate similarity

        if similarity < threshold: 
            continue
        
        graph.add_edge((pathway_1,pathway_resource_1),(pathway_2,pathway_resource_2), similarity=similarity)
        
        
    return graph



In [21]:
for i in range(1,11):
    
    pathway_similarity_network = nx.Graph()
    
    threshold = i/10
    
    print("#######################")
    print(threshold)
    print("#######################")

#     # Populate KEGG vs KEGG
#     pathway_similarity_network = populate_similarity_network(
#         pathway_similarity_network, 
#         threshold,
#         KEGG, kegg_pathways,
#         KEGG, kegg_pathways
#     )

    # Populate Reactome vs Reactome
    pathway_similarity_network = populate_similarity_network(
        pathway_similarity_network, 
        threshold,
        REACTOME, reactome_pathways,
        REACTOME, reactome_pathways
    )

#     # Populate WikiPathways vs WikiPathways
#     pathway_similarity_network = populate_similarity_network(
#         pathway_similarity_network, 
#         threshold,
#         WIKIPATHWAYS, wikipathways_pathways,
#         WIKIPATHWAYS, wikipathways_pathways
#     )

# # Populate Reactome vs KEGG
# pathway_similarity_network = populate_similarity_network(
#     pathway_similarity_network, 
#     threshold,
#     KEGG, kegg_pathways,
#     REACTOME, reactome_pathways
# )

# # Populate KEGG vs WikiPathways
# pathway_similarity_network = populate_similarity_network(
#     pathway_similarity_network, 
#     threshold,
#     KEGG, kegg_pathways,
#     WIKIPATHWAYS, wikipathways_pathways
# )

# # Populate Reactome vs WikiPathways
# pathway_similarity_network = populate_similarity_network(
#     pathway_similarity_network, 
#     threshold,
#     KEGG, kegg_pathways,
#     WIKIPATHWAYS, wikipathways_pathways
# )

    json_data = export_cytoscape_js(pathway_similarity_network)

    with open(REACTOME +'_{}'.format(i)+'0.json', 'w') as outfile:  
        json.dump(json_data, outfile)


#######################
0.1
#######################
#######################
0.2
#######################
#######################
0.3
#######################
#######################
0.4
#######################
#######################
0.5
#######################
#######################
0.6
#######################
#######################
0.7
#######################
#######################
0.8
#######################
#######################
0.9
#######################
#######################
1.0
#######################


In [ ]:
# print(json.dumps(json_data, indent=2))